In [2]:
from os import makedirs, path
from atlasreader import get_statmap_info
from IPython.display import display
from nilearn import image, plotting, reporting
from scipy import stats
from scipy.stats import pearsonr
from glob import glob
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns
import os, fnmatch

import re


The Python package you are importing, AtlasReader, is licensed under the
BSD-3 license; however, the atlases it uses are separately licensed under more
restrictive frameworks.
By using AtlasReader, you agree to abide by the license terms of the
individual atlases. Information on these terms can be found online at:
https://github.com/miykael/atlasreader/tree/master/atlasreader/data



In [3]:
# 定义绘制表格的函数
def combined_cluster_table(
    img_files_z=[],  # 用于 z-score 分析的图像文件列表
    img_files_ale=[], # 用于 ALE 分析的图像文件列表
    stub_keys=[], # 标记图像来源的标签列表
    stub_colname="Analysis",
    atlas="harvard_oxford", # 用于分区的图谱名称
    td_jar=None,
    output_file="cluster_table.tsv",
):

    # Create output director
    # 获取文件路径的目录名并创建目录，如果已经存在则忽略
    output_dir = path.dirname(output_file)
    makedirs(output_dir, exist_ok=True)

    # Create a list of DataFrames with peak and cluster stats for each image
    df_tuples = [
        get_statmap_info(img_file, cluster_extent=0, atlas="harvard_oxford", voxel_thresh=0)
        for img_file in img_files_z
    ]
    dfs = [
        pd.DataFrame(
            {
                "Cluster #": df_tuple[0]["cluster_id"],
                "Size (mm3)": df_tuple[0]["volume_mm"],
                "Cluster labels": df_tuple[0][atlas],
                "Mean z": df_tuple[0]["cluster_mean"],
                "Peak z": df_tuple[1]["peak_value"],
                "Peak X": df_tuple[1]["peak_x"],
                "Peak Y": df_tuple[1]["peak_y"],
                "Peak Z": df_tuple[1]["peak_z"],
                "Peak label": df_tuple[1][atlas],
            }
        )
        for df_tuple in df_tuples
    ]

    # Add ALE values if available
    if img_files_ale:
        df_tuples_ale = [
            get_statmap_info(img_file, cluster_extent=0, atlas="harvard_oxford", voxel_thresh=0)
            if img_file
            else (
                pd.DataFrame({"cluster_mean": [float("nan")]}),
                pd.DataFrame({"peak_value": [float("nan")]}),
            )
            for img_file in img_files_ale
        ]
        dfs_ale = [
            pd.DataFrame(
                {
                    "Mean ALE": df_tuple[0]["cluster_mean"],
                    "Peak ALE": df_tuple[1]["peak_value"],
                }
            )
            for df_tuple in df_tuples_ale
        ]
        for df, df_ale in zip(dfs, dfs_ale):
            df.insert(4, column="Mean ALE", value=df_ale["Mean ALE"])
            df.insert(6, column="Peak ALE", value=df_ale["Peak ALE"])

    # Concatenate into one big DataFrame
    df = pd.concat(dfs, keys=stub_keys)

    # Reformat numerical columns
    df["Size (mm3)"] = df["Size (mm3)"].apply(lambda x: "{:,.0f}".format(x))
    cols_int = ["Cluster #", "Peak X", "Peak Y", "Peak Z"]
    df[cols_int] = df[cols_int].applymap(int)
    cols_2f = ["Mean z", "Peak z"]
    df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
    if img_files_ale:
        cols_3f = ["Mean ALE", "Peak ALE"]
        df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))
        df[cols_3f] = df[cols_3f].replace("nan", "")

    # Add the stub column
    df.index = df.index.set_names(stub_colname, level=0)
    df.reset_index(level=stub_colname, inplace=True)
    mask = df[stub_colname].duplicated()
    df.loc[mask.values, [stub_colname]] = ""

    # Save to CSV
    df.to_csv(output_file, sep="\t", index=False)

    return df

In [3]:
tab1 = combined_cluster_table(
    img_files_z=[
        "../results/ale/Self_all_z_size_level_thresh.nii.gz",
    ],
    stub_keys=[
        "Activation likelihood estimation",
    ],
    stub_colname="Analysis",
    img_files_ale=[
        "../results/ale/Self_all_stat_size_thresh.nii.gz"
    ],
    atlas="harvard_oxford",
    output_file="../results/tables/tab_self_all.tsv",
)
display(tab1)

/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_24282/3404049349.py:69: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_int] = df[cols_int].applymap(int)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_24282/3404049349.py:71: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_24282/3404049349.py:74: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))


,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,"24,336",28.34% Left_Frontal_Pole; 21.33% Left_Paracing...,3.54,0.027,3.54,0.056,-6,46,12,57.0% Left_Paracingulate_Gyrus; 30.0% Left_Cin...
1,,2,"5,864",69.03% Left_Frontal_Orbital_Cortex; 16.78% Lef...,3.54,0.024,3.54,0.044,-40,22,-12,72.0% Left_Frontal_Orbital_Cortex
2,,3,"3,728",46.14% Left_Cingulate_Gyrus_posterior_division...,3.54,0.029,3.54,0.065,-6,-54,26,49.0% Left_Cingulate_Gyrus_posterior_division;...
3,,4,"1,680",44.29% Left_Cingulate_Gyrus_posterior_division...,3.54,0.027,3.54,0.046,-2,-18,36,75.0% Left_Cingulate_Gyrus_posterior_division;...
4,,5,"1,616",86.14% Left_Angular_Gyrus; 13.86% Left_Lateral...,3.54,0.026,3.54,0.048,-46,-58,24,60.0% Left_Angular_Gyrus; 19.0% Left_Lateral_O...


In [4]:
tab2 = combined_cluster_table(
    img_files_z=[
        "../results/ale/Self_st_z_size_level_thresh.nii.gz",
    ],
    stub_keys=[
        "Activation likelihood estimation",
    ],
    stub_colname="Analysis",
    img_files_ale=[
        "../results/ale/Self_st_stat_size_thresh.nii.gz"
    ],
    atlas="harvard_oxford",
    output_file="../results/tables/tab_self_st.tsv",
)
display(tab2)

/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_71475/3404049349.py:69: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_int] = df[cols_int].applymap(int)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_71475/3404049349.py:71: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_71475/3404049349.py:74: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))


,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,"18,504",27.63% Left_Paracingulate_Gyrus; 18.20% Left_C...,3.54,0.025,3.54,0.050,-4,46,8,54.0% Left_Paracingulate_Gyrus; 41.0% Left_Cin...
1,,2,"4,384",75.36% Left_Frontal_Orbital_Cortex; 7.30% Left...,3.54,0.023,3.54,0.044,-42,24,-12,60.0% Left_Frontal_Orbital_Cortex
2,,3,"3,888",83.33% Left_Frontal_Pole; 16.67% Left_Superior...,3.54,0.020,3.54,0.031,-20,44,38,78.0% Left_Frontal_Pole
3,,4,"2,672",55.39% Left_Cingulate_Gyrus_posterior_division...,3.54,0.024,3.54,0.049,-6,-54,26,49.0% Left_Cingulate_Gyrus_posterior_division;...
4,,5,"1,168",92.47% Left_Angular_Gyrus; 7.53% Left_Lateral_...,2.43,0.022,2.43,0.035,-46,-58,24,60.0% Left_Angular_Gyrus; 19.0% Left_Lateral_O...
5,,6,"1,128",44.68% Left_Cingulate_Gyrus_posterior_division...,2.36,0.022,2.36,0.035,-2,-18,38,73.0% Left_Cingulate_Gyrus_posterior_division;...


In [4]:
tab3 = combined_cluster_table(
    img_files_z=[
        "../results/ale/unhealth_z_size_level_thresh.nii.gz",
    ],
    stub_keys=[
        "Activation likelihood estimation",
    ],
    stub_colname="Analysis",
    img_files_ale=[
        "../results/ale/unhealth_stat_size_thresh.nii.gz"
    ],
    atlas="harvard_oxford",
    output_file="../results/tables/tab_self_unhealth.tsv",
)
display(tab3)

/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_94084/3404049349.py:69: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_int] = df[cols_int].applymap(int)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_94084/3404049349.py:71: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_94084/3404049349.py:74: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))


,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,"1,728",42.13% Left_Precuneous_Cortex; 34.72% Left_Cin...,3.35,0.012,3.35,0.018,-4,-54,28,56.0% Left_Cingulate_Gyrus_posterior_division;...
1,,2,"1,288",85.71% Left_Frontal_Pole; 8.70% Right_Frontal_...,3.04,0.012,3.04,0.020,-6,58,16,62.0% Left_Frontal_Pole; 12.0% Left_Paracingul...
